<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/model_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers

In [0]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Dense, Activation, Concatenate, Dropout
from transformers import TFBertModel

print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-dev20191226


In [0]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['192.168.60.2:8470']


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: srihari-tpu


INFO:tensorflow:Initializing the TPU system: srihari-tpu


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:5, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:5, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


In [0]:
batch_size = 12 * strategy.num_replicas_in_sync
embedding_dim = 512
autotune = tf.data.experimental.AUTOTUNE

train_steps = 1262996 * 0.8 // batch_size
val_steps = train_steps // 10
epochs = 100

In [0]:
base_dir = 'gs://tfworld'
model_dir = os.path.join(base_dir, 'model_files')
tensorboard_dir = os.path.join(base_dir, 'model_files', 'logs')
tfrecords_pattern_train = os.path.join(base_dir, 'tfrecords', 'train*')
tfrecords_pattern_val = os.path.join(base_dir, 'tfrecords', 'eval*')


In [0]:
features = {
    'title':tf.io.VarLenFeature(dtype=tf.int64),
    'citation':tf.io.FixedLenFeature([512], dtype=tf.float32),
    }

def parse_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, features)
    title = tf.sparse.to_dense(parsed_example['title'])
    citation = parsed_example['citation']
    
    title = tf.cast(title, dtype=tf.int32)
    citation = tf.cast(citation, dtype=tf.float32)
    title = title[:512]
    title = tf.pad(title, paddings=[[0, 512-tf.shape(title)[0]]])
    return (title, citation), tf.constant([1.0], dtype=tf.float32)

In [0]:
with strategy.scope():
    train_files = tf.data.Dataset.list_files(tfrecords_pattern_train)
    train_dataset = train_files.interleave(tf.data.TFRecordDataset,
                                           cycle_length=16,
                                           block_length=4,
                                           num_parallel_calls=autotune)
    train_dataset = train_dataset.map(parse_example, num_parallel_calls=autotune)
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
    train_dataset = train_dataset.prefetch(autotune)

    val_files = tf.data.Dataset.list_files(tfrecords_pattern_val)
    val_dataset = val_files.interleave(tf.data.TFRecordDataset,
                                       cycle_length=16,
                                       block_length=4,
                                       num_parallel_calls=autotune)
    val_dataset = val_dataset.map(parse_example, num_parallel_calls=autotune)
    val_dataset = val_dataset.batch(batch_size, drop_remainder=True)
    val_dataset = val_dataset.prefetch(autotune)

tf.data.experimental.get_structure(train_dataset), tf.data.experimental.get_structure(val_dataset)

((TensorSpec(shape=(96, None), dtype=tf.int32, name=None),
  TensorSpec(shape=(96, 512), dtype=tf.float32, name=None)),
 TensorSpec(shape=(96, 1), dtype=tf.float32, name=None))

In [0]:
@tf.function
def loss_fn(_, probs):
    '''
        1. Every sample is its own positive, and  the rest of the
            elements in the batch are its negative.
        2. Each TPU core gets 1/8 * global_batch_size elements, hence
            compute shape dynamically.
        3. Dataset produces dummy labels to make sure the loss_fn matches
            the loss signature of keras, actual labels are computed inside this
            function.
        4. `probs` lie in [0, 1] and are to be treated as probabilities.
    '''
    bs = tf.shape(probs)[0] 
    labels = tf.eye(bs, bs)
    return tf.losses.categorical_crossentropy(labels, probs, from_logits=False)
    
def create_model(drop_out):
    textIds = tf.keras.Input(shape=(512,), dtype=tf.int32)    # from bert tokenizer
    citation = tf.keras.Input(shape=(512,))                   # normalized word2vec outputs
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    textOut = bert_model(textIds)
    textOutMean = tf.reduce_mean(textOut[0], axis=1)
    textOutSim = Dense(units=embedding_dim, activation='tanh', name='DenseTitle')(textOutMean)
    textOutSim = Dropout(drop_out)(textOutSim)
    
    citationSim = Dense(units=embedding_dim, activation='tanh', name='DenseCitation')(citation)
    citationSim = Dropout(drop_out)(citationSim)

    # Get dot product of each of title x citation combinations
    dotProduct = tf.reduce_sum(tf.multiply(textOutSim[:, None, :], citationSim), axis=-1)
    
    # Softmax to make sure each row has sum == 1.0
    probs = tf.nn.softmax(dotProduct, axis=-1)

    model = tf.keras.Model(inputs=[textIds, citation], outputs=[probs])
    return model

In [0]:
with strategy.scope():
    model = create_model(drop_out=.1)
    model.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5))

In [0]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=tensorboard_dir, update_freq='epoch'), 
            tf.keras.callbacks.ModelCheckpoint(filepath=model_dir + '/{epoch:02d}_{loss:.2f})',
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_freq='epoch')
                                               ]

In [0]:
model.fit(dataset,
          epochs=epochs,
          steps_per_epoch=train_steps,
          validation_data=val_dataset,
          validation_steps=val_steps,
          validation_freq=2
          callbacks=callbacks)

Train for 13156 steps


Epoch 1/100


    6/13156 [..............................] - ETA: 75:25:08 - loss: 2.5508
Epoch 00001: saving model to gs://tfworld/model_files/01-2.55)
    6/13156 [..............................] - ETA: 76:22:36 - loss: 2.5508

KeyboardInterrupt: 